In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing import image_dataset_from_directory
from keras.applications import MobileNetV3Small
from keras.applications.mobilenet_v3 import preprocess_input


In [2]:
#Loading data

train_data = image_dataset_from_directory(
    directory="../data/SkinDisease/train",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(244,244),
    batch_size=32

)

test_data = image_dataset_from_directory(
    directory="../data/SkinDisease/test",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(244,244),
    batch_size=32,
    shuffle=False
)

Found 13898 files belonging to 22 classes.
Found 1546 files belonging to 22 classes.


In [3]:
# Preprocessing images
def preprocess(image, label):
    image = tf.image.resize(image, (224,224))
    image = preprocess_input(image)

    return image, label

train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)


In [4]:
train_data = train_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [5]:
augment_data = Sequential([
    
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomFlip('vertical'),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomRotation(0.05)

])

In [6]:
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [7]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

In [8]:
print(len(base_model.layers))

157


In [9]:
# Building model
model = Sequential()

model.add(keras.layers.Input(shape=(224,224,3)))
model.add(augment_data)
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.12))

model.add(Dense(22, activation='softmax'))

In [10]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 22)             │         2,838 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,015,814 (3.88 MB)

 Trainable params: 664,886 (2.54 MB)

 Non-trainable params: 350,928 (1.34 MB)

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, epochs=20, validation_data=test_data)

Epoch 1/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 274s 582ms/step - accuracy: 0.3940 - loss: 1.9981 - val_accuracy: 0.3726 - val_loss: 2.2450
Epoch 2/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 231s 530ms/step - accuracy: 0.5297 - loss: 1.5212 - val_accuracy: 0.4521 - val_loss: 2.1680
Epoch 3/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 223s 512ms/step - accuracy: 0.5920 - loss: 1.2914 - val_accuracy: 0.4974 - val_loss: 1.8654
Epoch 4/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 227s 521ms/step - accuracy: 0.6462 - loss: 1.1098 - val_accuracy: 0.5388 - val_loss: 1.8216
Epoch 5/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 222s 511ms/step - accuracy: 0.6967 - loss: 0.9508 - val_accuracy: 0.5712 - val_loss: 1.8422
Epoch 6/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 219s 505ms/step - accuracy: 0.7302 - loss: 0.8384 - val_accuracy: 0.5634 - val_loss: 1.8471
Epoch 7/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 235s 540ms/step - accuracy: 0.7699 - loss: 0.7157 - val_accuracy: 0.5841 - val_loss: 1.9112
Epoch 8/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 229s 526ms/step - accuracy: 0.7941 -